<a href="https://colab.research.google.com/github/linyu2703/nhanes_inferential_2021_23/blob/main/inferential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# importing libraries
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt

In [3]:
# loading and reading datasets
demo_src = pd.read_sas('DEMO_L.xpt')
bpxo_src = pd.read_sas('BPXO_L.xpt')
hepb_src = pd.read_sas('HEPB_S_L.xpt')
vid_src = pd.read_sas('VID_L.xpt')
kiq_src = pd.read_sas('KIQ_U_L.xpt')
paq_src = pd.read_sas('PAQ_L.xpt')
whq_src = pd.read_sas('WHQ_L.xpt')

In [4]:
# creating a separate dataframe from the main src dataframe with specific columns
# creating a main dataframe with those dataframes merged by SEQN along with the other specified columns
demo = demo_src[['SEQN','DMDMARTZ','DMDEDUC2','RIDAGEYR']].copy()
bpxo = bpxo_src[['SEQN','BPXOSY3','BPXODI3']].copy()
hepb = hepb_src[['SEQN', 'LBXHBS']].copy()
vid = vid_src[['SEQN', 'LBDVD2LC']].copy()
kiq = kiq_src[['SEQN', 'KIQ022']].copy()
paq = paq_src[['SEQN', 'PAD680']].copy()
whq = whq_src[['SEQN', 'WHD020']].copy()
df = demo.copy()
df = (demo
      .merge(bpxo, on='SEQN', how='left')
      .merge(hepb, on='SEQN', how='left')
      .merge(vid, on='SEQN', how='left')
      .merge(kiq, on='SEQN', how='left')
      .merge(paq, on='SEQN', how='left')
      .merge(whq, on='SEQN', how='left')
      )

In [5]:
# renaming columns and recoding values for clarity
df_cleaned = df[['SEQN','DMDMARTZ','DMDEDUC2','RIDAGEYR','BPXOSY3','BPXODI3','LBXHBS','LBDVD2LC','KIQ022','PAD680','WHD020']].copy().rename(columns={
    'SEQN': 'ID',
    'DMDMARTZ': 'Marital Status',
    'DMDEDUC2': 'Education Level',
    'RIDAGEYR': 'Age',
    'BPXOSY3': 'Systolic BP',
    'BPXODI3': 'Diastolic BP',
    'LBXHBS': 'Hep B',
    'LBDVD2LC': 'Vitamin D',
    'KIQ022': 'Kidney Condition',
    'PAD680': 'Physical Activity',
    'WHD020': 'Weight'
})

df_cleaned['Marital Status'] = np.where(df_cleaned['Marital Status'] == 1, 'married',
                                   np.where(df_cleaned['Marital Status'].isin([2,3,77,99]), 'not married', pd.NA))
df_cleaned['Education Level'] = np.where(df_cleaned['Education Level'].isin([4,5]),'bachelors or higher',
                                    np.where(df_cleaned['Education Level'].isin([1,2,3,7,9]),'less than bachelors', pd.NA))
df_cleaned['Age'] = df_cleaned['Age'].mask(df_cleaned['Age'].abs() < 1e-10, np.nan)
df_cleaned['Kidney Condition'] = np.where(df_cleaned['Kidney Condition'] == 1, 'Yes', np.where(df_cleaned['Kidney Condition'] == 2, 'No', pd.NA))
df_cleaned['Physical Activity'] = df_cleaned['Physical Activity'].replace([7777,9999], np.nan)
df_cleaned['Weight'] = df_cleaned['Weight'].replace([7777,9999], np.nan)

df_cleaned['Physical Activity'] = df_cleaned['Physical Activity'].fillna(round(df_cleaned['Physical Activity'].median(), 0))
df_cleaned['Weight'] = df_cleaned['Weight'].fillna(round(df_cleaned['Weight'].mean(), 0))

In [6]:
# Question 1: "Is there an association between marital status (married or not married) and education level (bachelor’s degree or higher vs. less than a bachelor’s degree)?"
# chi-squared test will be used to test whether the two variables: (martial status and education level) are independent or dependent
Q1 = df_cleaned.loc[(df_cleaned['Marital Status'].isin(['married','not married'])) &
    (df_cleaned['Education Level'].isin(['bachelors or higher','less than bachelors']))
].copy()

Q1['Marital Status'].value_counts()
Q1['Education Level'].value_counts()

ct = pd.crosstab(Q1['Marital Status'], Q1['Education Level'])
print(ct)

# Performing chi-square analysis
chi2, p, _, _ = chi2_contingency(ct)
print("Chi-square =", round(chi2, 2))
print("p value =", p)


Education Level  bachelors or higher  less than bachelors
Marital Status                                           
married                         2782                 1354
not married                     2212                 1444
Chi-square = 38.24
p value = 6.264475724779772e-10


p value is less than 0.05, which means there is a significant correlation between martial status and education level. people with bachelors degree or higher are more likely to be married compared to those with less than bachelors degree.

In [7]:
# Question 2: "Is there a difference in the mean sedentary behavior time between those who are married and those who are not married?"
# 2 sample t-test will be performed to test the relationship between maritial status and physical actvity
# null hypothesis: no difference in mean sedentary time between married and not married people
# alternative hypothesis: there is a difference in mean sedentary time between married and not married people
Q2 = df_cleaned.loc[
    (df_cleaned['Marital Status'].isin(['married','not married'])) &
    (df_cleaned['Physical Activity'] != 'NaN')
].copy()

married = Q2.loc[Q2['Marital Status']=='married', 'Physical Activity']
not_married = Q2.loc[Q2['Marital Status']=='not married', 'Physical Activity']

t_stat, p_val = ttest_ind(married, not_married, equal_var=False)
print("t-statistic =", round(t_stat, 2))
print("p value =", p_val )

married_mean = married.mean()
not_married_mean = not_married.mean()
print("Married Mean =", round(married_mean, 2))
print("Not Married Mean =", round(not_married_mean, 2))

t-statistic = -3.78
p value = 0.00015846852964065343
Married Mean = 352.9
Not Married Mean = 371.03


p value is less than 0.05 which means there is a correlation between martial status and physical activity, the null hypothesis is rejected. the t-stat is -3.78 meaning married people have a lowered mean than not married people, so married people are more likely to be less sedentary compared to than those who are not married.

In [12]:
# Question 3: "How do age and marital status affect systolic blood pressure?"
# 2 way ANOVA test will be used to test the relationship between marital status and age with systolic blood pressure
Q3 = df_cleaned.loc[
    (df_cleaned['Marital Status'].isin(['married', 'not married'])) &
    (df_cleaned['Age'] != 'NaN') &
    (df_cleaned['Systolic BP'] != 'NaN')
].copy()

Q3 = Q3.rename(columns={
    'Systolic BP': 'Systolic_BP',
    'Marital Status': 'Marital_Status',
})

model = ols('Systolic_BP ~ Marital_Status + Age', data=Q3).fit()

anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)


                      sum_sq      df           F         PR(>F)
Marital_Status  2.535753e+03     1.0    8.678543   3.232503e-03
Age             2.652276e+05     1.0  907.734014  1.738958e-185
Residual        1.706077e+06  5839.0         NaN            NaN


For age: f = 907.73 and p value less than 0.0000000->17 which is extremely small, showing a strong correlation between age and systolic blood pressure. so as people age their systolic blood pressure tends to rise significantly.
For martial status: f = 8.68 and p value is 0.0032 which is also relatively small, showing that martial status influences systolic blood pressure as well. married and not married individuals have different avg systolic blood pressure even when controlling for age.

In [18]:
# Question 4: "Is there a correlation between self-reported weight and minutes of sedentary behavior?"
# linear regression test will be used to test whether weight predicts sedentary behavior
Q4 = df_cleaned.loc[
    (df_cleaned['Weight'] != 'NaN') &
    (df_cleaned['Physical Activity'] != 'NaN')
].copy()

weight = Q4['Weight']
physical_activity = Q4['Physical Activity']

weight = sm.add_constant(weight)

model = sm.OLS(physical_activity, weight).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:      Physical Activity   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     291.8
Date:                Tue, 28 Oct 2025   Prob (F-statistic):           1.17e-64
Time:                        22:50:51   Log-Likelihood:                -78425.
No. Observations:               11933   AIC:                         1.569e+05
Df Residuals:                   11931   BIC:                         1.569e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        221.3187      7.241     30.564      0.0

Coefficient is  = 0.67 and p = 0.000, which is less than 0.001, this means that there is correlation between weight and physical activity. the higher the weight of an individual the less time being spent on sedentary. the r^2 = 0.024, which means weight alone does not strongly influence the sedentary behavior.

In [26]:
# Question 5: Develop your own unique question using at least one of the variables listed above.
# t-test will be used to test the relationship between kidney condition (individuals with and without) and age
Q5 = df_cleaned.loc[
    (df_cleaned['Kidney Condition'].isin(['Yes', 'No'])) &
    (df_cleaned['Age'] != 'NaN')
].copy()

kidney_yes = Q5.loc[Q5['Kidney Condition'] == 'Yes', 'Age']
kidney_no = Q5.loc[Q5['Kidney Condition'] == 'No', 'Age']

t_stat, p_val = ttest_ind(kidney_yes, kidney_no, equal_var=False)
print("t-statistic =", round(t_stat, 2))
print("p-value =", p_val)

print("Yes Kidney Condition Average Age =", round(kidney_yes.mean(), 2))
print("No Kidney Condition Average Age =", round(kidney_no.mean(), 2))

t-statistic = 16.01
p-value = 4.456766612897944e-44
Yes Kidney Condition Average Age = 65.81
No Kidney Condition Average Age = 53.08


t-statistic value is 16.01 and p value is 0.000000...4, less than 0.001, shows that there is a correlation between avg age and individuals with and without kidney conditions. People with kidney conditions with a mean of 65.81 yrs are older than those without kidney conditions with a mean of 53.1 yrs.